# Project 1, Part 4, Holiday Related Queries



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.4.1 For each day in 2020, find the day of week 

The example below shows you how to generate every day in 2020.  Use it as your starter SQL:

```
select my_date::date as date_analyzed
from generate_series('2020-01-01', '2020-12-31', '1 day'::interval) as my_date
order by date_analyzed
```

Derive the dow (0 = Sunday) and the day_of_week from the date_analyzed.

Sort by date_analyzed.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**

The query should return 366 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||date_analyzed|dow|day_of_week|
|---|---|---|---|
|0|2020-01-01|3|Wednesday|
|...|...|...|...|
|365|2020-12-31|4|Thursday |

In [4]:
rollback_before_flag = True
rollback_after_flag = True

query = """

/*
Generate every day in 2020 using the generate_series function.
*/

select 
    /*
    Get the generated date as date_analyzed.
    */
    my_date::date as date_analyzed,
    
    /*
    Extract the day of week and the day name from the date_analyzed.
    */
    extract(dow from my_date)::integer as dow,

    trim(to_char(my_date, 'Day')) as day_of_week

from 
    /*
    Generate every day in 2020 using the generate_series function.
    */
    generate_series('2020-01-01', '2020-12-31', '1 day'::interval) as my_date

/*
Sort the results by the date_analyzed.
*/
order by 
    date_analyzed;
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,date_analyzed,dow,day_of_week
0,2020-01-01,3,Wednesday
1,2020-01-02,4,Thursday
2,2020-01-03,5,Friday
3,2020-01-04,6,Saturday
4,2020-01-05,0,Sunday
...,...,...,...
361,2020-12-27,0,Sunday
362,2020-12-28,1,Monday
363,2020-12-29,2,Tuesday
364,2020-12-30,3,Wednesday


# 1.4.2 For each holiday in 2020, find dates within a 7 day window and also display the day of week

Using the query from the 1.4.1 as a starting point, join to the holidays table where my_date is between holiday_date - 7 and holiday_date + 7 to find a 7 day window, limited to dates in 2020. 

Use the description in the holidays table for the holiday_name.

Derive the dow (0 = Sunday) and the day_of_week from the date_analyzed.

Sort by date_analyzed.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**

The query should return 172 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||holiday_name|date_analyzed|dow|day_of_week|
|---|---|---|---|---|
|0|New Year's Day|2020-01-01|3|Wednesday|
|...|...|...|...|...|
|171|Christmas|2020-12-31|4|Thursday |


In [5]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
    /*
    use the description in the holidays table for the holiday_name.
    */
    holidays.description as holiday_name,
    
    /*
    keep the date as date_analyzed.
    */
    my_date::date as date_analyzed,
    
    /*
    derive the day of the week number (dow) and day of the week name (day_of_week) 
    from the date_analyzed.
    */
    extract(dow from my_date)::integer as dow,
    trim(to_char(my_date, 'Day')) as day_of_week

from 
    /*
    generate each day of 2020.
    */
    generate_series('2020-01-01', '2020-12-31', '1 day'::interval) as my_date, holidays
    
    /*
    join with the holidays table where the date is within a 7 day window of the holiday.
    */
--    join holidays on my_date = holiday_date;
--    join holidays on 
--    	my_date::date between holidays.holiday_date - interval '7 days'
--    	and holidays.holiday_date + interval '7 days'
/*
Include only the dates that fall within a 7 day window of each holiday.
*/
where 
    my_date::date between holidays.holiday_date - interval '7 days'
    and holidays.holiday_date + interval '7 days'

/*
sort the results by date_analyzed.
*/
order by 
    date_analyzed;
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,holiday_name,date_analyzed,dow,day_of_week
0,New Year's Day,2020-01-01,3,Wednesday
1,New Year's Day,2020-01-02,4,Thursday
2,New Year's Day,2020-01-03,5,Friday
3,New Year's Day,2020-01-04,6,Saturday
4,New Year's Day,2020-01-05,0,Sunday
...,...,...,...,...
167,Christmas,2020-12-27,0,Sunday
168,Christmas,2020-12-28,1,Monday
169,Christmas,2020-12-29,2,Tuesday
170,Christmas,2020-12-30,3,Wednesday


# 1.4.3 Sum the sales for each day that there were sales

Each record in the sales table is an individual sale, and the total_amount is the total amount for that individual sale.  There are only records in the sales table for day with sales, so days without sales will not be included.

Sort by sale_date.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**

The query should return 364 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||sale_date|sales_dollars|
|---|---|---|
|0|2020-01-01|133776|
|...|...|...|
|363|2020-12-31|266100|


In [6]:
rollback_before_flag = True
rollback_after_flag = True

query = """

/*
Starting from the sales table.
*/
select 
    /*
    Extracting the sale_date from the sales table.
    */
    sales.sale_date,

    /*
    Summing the total_amount for each sale_date to calculate total sales for each day.
    */
    sum(sales.total_amount) as sales_dollars

from 
    /*
    The base table for this operation is the sales table.
    */
    sales

/*
Grouping the results by sale_date to sum total_amount for each day.
*/
group by 
    sales.sale_date

/*
Ordering the final result by sale_date in ascending order.
*/
order by 
    sales.sale_date;

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,sale_date,sales_dollars
0,2020-01-01,133776
1,2020-01-02,261120
2,2020-01-03,263076
3,2020-01-04,400368
4,2020-01-05,394992
...,...,...
359,2020-12-27,396096
360,2020-12-28,264852
361,2020-12-29,130440
362,2020-12-30,263424


# 1.4.4 Find the expected sales by day of week

Use the query from 1.4.3, which returns a sale_date and sales_dollars, as a subquery.  When using as a subquery, remove its "order by" clause.

For the outer query, select from the subquery and aggregate by dow & day_of_week, finding the average of sales_dollars for each dow & day_of_week, which we will call expected_sales_dollars.  In the outer query, derive the dow (0 = Sunday) and the day_of_week from the sale_date in the subquery.

Sort by dow.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return 7 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||dow|day_of_week|expected_sales_dollars|
|---|---|---|---|
|0|0|Sunday   |357482|
|...|...|...|...|
|6|6|Saturday |373490|


In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """
          
/*
Starting from a subquery that gets the daily sales transactional data.
*/
with sales_trx_daily as (
    select 
        /*
        Extracting the sale_date from the sales table.
        */
        sales.sale_date,

        /*
        Summing the total_amount for each sale_date to calculate total sales for each day.
        */
        sum(sales.total_amount) as sales_dollars

    from 
        /*
        The base table for this operation is the sales table.
        */
        sales

    /*
    Grouping the results by sale_date to sum total_amount for each day.
    */
    group by 
        sales.sale_date
)

select 
    /*
    Extracting the day of week from sale_date.
    */
    extract(dow from sales_trx_daily.sale_date) as dow,

    /*
    Formatting the day of week from sale_date.
    */
    to_char(sales_trx_daily.sale_date, 'Day') as day_of_week,

    /*
    Calculating the average of sales_dollars for each day of the week.
    */
    round(avg(sales_trx_daily.sales_dollars),0) as expected_sales_dollars

from 
    /*
    Selecting from the subquery daily_sales.
    */
    sales_trx_daily

/*
Grouping the results by day of week to calculate the average sales.
*/
group by 
    extract(dow from sales_trx_daily.sale_date), to_char(sales_trx_daily.sale_date, 'Day')

/*
Ordering the final result by dow in ascending order.
*/
order by 
    dow;

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,dow,day_of_week,expected_sales_dollars
0,0,Sunday,357482
1,1,Monday,253225
2,2,Tuesday,132603
3,3,Wednesday,263256
4,4,Thursday,266051
5,5,Friday,252522
6,6,Saturday,373490


# 1.4.5 Find the actual sales for every day of 2020, even days AGM was closed

The query from 1.4.3 returns a sale_date and sales_dollars for days with sales.  Use the sales_dollars for actual_sales_dollars. It does not include days when AGM was closed and didn't have any sales.

In this query we want to also include a sale_date and 0 for the actual_sales_dollars for the days that AGM was closed.

In the holidays table the closed_flag tells us the days that AGM was closed and will not have any sales.

Consider using the query from 1.4.3 as a subquery, writing another subquery that finds the days that AGM is closed, and combines the rows from the 2 subqueries using a "union".

Sort by sale_date.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**

The query should return 366 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||sale_date|actual_sales_dollars|
|---|---|---|
|0|2020-01-01|133776|
|...|...|...|
|365|2020-12-31|266100|


In [12]:
rollback_before_flag = True
rollback_after_flag = True

query = """
          
/*
This  query combines two parts using the 'union all' operator.
Part 1: Calculate the total sales for each day when the store was open.
The 'sales' table is utilized here, grouping by the sale_date to get one record per day,
and summing the total_amount to calculate the total sales for that day.
*/
select 
    /* sale_date: The date of the sale */
    sales.sale_date as sale_date,
    
    /* actual_sales_dollars: The total sales for that day */
    sum(sales.total_amount) as actual_sales_dollars 
from 
    /* The 'sales' table is being used for this part of the query */
    sales
group by 
    /* Grouping by the sale_date to get one record per day */
    sales.sale_date 

/*
Combining the results of Part 1 and Part 2 using UNION ALL operator.
'union all' includes all records from both parts, even if there are duplicates.
*/

union all

/* 
Part 2: Include the days when the store was closed.
On these days, 0 is used as the total sales, because the store was closed and there 
were no sales.
The 'holidays' table is used for this, and only records where the closed_flag is true 
are included.
*/

select 
    /* sale_date: The date of the holiday */
    holidays.holiday_date as sale_date,
    
    /* actual_sales_dollars: The total sales for that day, which is 0 because the store 
    was closed */
    0 as actual_sales_dollars 
from 
    /* The 'holidays' table is used for this part of the query */
    holidays
where 
    /* Only records where the store was closed are included */
    holidays.closed_flag = true 

/*
Finally, the results are ordered by the sale_date.
*/
order by 
    sale_date;          
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,sale_date,actual_sales_dollars
0,2020-01-01,133776
1,2020-01-02,261120
2,2020-01-03,263076
3,2020-01-04,400368
4,2020-01-05,394992
...,...,...
361,2020-12-27,396096
362,2020-12-28,264852
363,2020-12-29,130440
364,2020-12-30,263424


# 1.4.6  Find holiday days, and days within a one week of a holiday, where the actual sales are reduced by 15% or more from expected sales

Now that we have some preliminary queries written, we can use them as subqueries to a larger query.  Remember to remove the "order by" clause when using them as subqueries.

The query in 1.4.2 returns the holiday_name, date_analyzed, dow, and day_of_week.

The query in 1.4.4 returns the dow, day_of_week, and expected_sales_dollars.

The query in 1.4.5 returns the sale_date and actual_sales_dollars for all days of 2020, even days where AGM was closed.

Join the 3 subqueries together to return holiday_name, date_analyzed, day_of_week, actual_sales_dollars, and expected_sales_dollars.

Derive a column ratio_actual_expected by dividing actual_sales_dollars by the expected_sales_dollars, rounded as shown.

Filter where the ratio_actual_expected (unrounded) is 85% or less.

Sort by date_analyzed

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

**Note: the reference output is in Markdown which drops trailing zeros.  Pandas does not drop trailing zeros.  This is ok.**

The query should return 26 rows into a Pandas dataframe and should look similar to this  (note that markdown drops the trailing zero while Pandas does not - this is fine): 

||holiday_name|date_analyzed|day_of_week|actual_sales_dollars|expected_sales_dollars|ratio_actual_expected|
|---|---|---|---|---|---|---|
|0|New Year's Day|2020-01-01|Wednesday|133776|263256|0.51|
|1|MLK Day|2020-01-17|Friday   |127092|252522|0.5|
|2|MLK Day|2020-01-18|Saturday |135204|373490|0.36|
|3|MLK Day|2020-01-19|Sunday   |130368|357482|0.36|
|4|MLK Day|2020-01-20|Monday   |130740|253225|0.52|
|5|President's Day|2020-02-14|Friday   |133452|252522|0.53|
|6|President's Day|2020-02-15|Saturday |132096|373490|0.35|
|7|President's Day|2020-02-16|Sunday   |132180|357482|0.37|
|8|President's Day|2020-02-17|Monday   |135228|253225|0.53|
|9|Easter|2020-04-12|Sunday   |136164|357482|0.38|
|10|Mother's Day|2020-05-10|Sunday   |134496|357482|0.38|
|11|Memorial Day|2020-05-22|Friday   |129132|252522|0.51|
|12|Memorial Day|2020-05-23|Saturday |132900|373490|0.36|
|13|Memorial Day|2020-05-24|Sunday   |130764|357482|0.37|
|14|Memorial Day|2020-05-25|Monday   |132624|253225|0.52|
|15|Father's Day|2020-06-21|Sunday   |135120|357482|0.38|
|16|Independence Day|2020-07-03|Friday   |132960|252522|0.53|
|17|Independence Day|2020-07-04|Saturday |134952|373490|0.36|
|18|Independence Day|2020-07-05|Sunday   |133884|357482|0.37|
|19|Independence Day|2020-07-06|Monday   |134964|253225|0.53|
|20|Labor Day|2020-09-04|Friday   |132048|252522|0.52|
|21|Labor Day|2020-09-05|Saturday |132840|373490|0.36|
|22|Labor Day|2020-09-06|Sunday   |133332|357482|0.37|
|23|Labor Day|2020-09-07|Monday   |130020|253225|0.51|
|24|Thanksgiving|2020-11-26|Thursday |0|266051|0.0|
|25|Christmas|2020-12-25|Friday   |0|252522|0.0|


In [21]:
rollback_before_flag = True
rollback_after_flag = True

query = """

/*
Purpose: Find holiday days, and days within a one-week window around a holiday,
where the actual sales are reduced by 15% or more from the expected sales.
*/

/* Temporary table for holidays and days within a one-week window of a holiday */
with holidays_tmp as (
    /*
    Extracting the holiday dates, the holiday names, and other necessary details 
    from the holidays table. We only focus on the year 2020.
    Extending the range to include a one-week window around each holiday.
    */
    select 
        generate_series(holiday_date - interval '7 day', holiday_date + interval '7 day', 
        interval '1 day')::date AS date_analyzed, description AS holiday_name,
        extract(dow from generate_series(holiday_date - interval '7 day', 
        holiday_date + interval '7 day', interval '1 day'))::int as dow,
        to_char(generate_series(holiday_date - interval '7 day', 
        holiday_date + interval '7 day', interval '1 day'), 'Day') as day_of_week
    from 
        holidays
    where
        holiday_date between '2020-01-01' and '2020-12-31'
),

/* Temporary table for actual sales dollars */
actual_sales_tmp as (
    /*
    Combining the two parts using the 'union all' operator.
    Part 1: Calculate the total sales for each day when the store was open.
    */
    select 
        sales.sale_date,
        sum(sales.total_amount) as actual_sales_dollars 
    from 
        sales
    group by 
        sales.sale_date

    union all

    /*
    Part 2: Include the days when the store was closed (holidays with zero actual sales).
    */
    select 
        holidays.holiday_date,
        0 as actual_sales_dollars 
    from 
        holidays
    where 
        holidays.closed_flag = true
),

/* Temporary table for expected sales dollars */
expected_sales_tmp as (
    /*
    For each day of the week, we calculate the average total amount from the sales table.
    This gives the expected sales dollars for each day of the week.
    */
    with sales_trx_daily AS (
        select 
            sale_date,
            SUM(total_amount) AS daily_total_amount
        from 
            sales
        group by 
            sale_date
    )
    select 
        extract(dow FROM sale_date)::int as dow,
        to_char(sale_date, 'Day') as day_of_week,
        round(avg(daily_total_amount), 0) AS expected_sales_dollars
    from 
        sales_trx_daily
    group by 
        dow, day_of_week
)

/* Main query to join all temp tables and calculate ratio */
/*
We are now joining our temporary tables together and calculating the ratio of actual 
sales to expected sales. We are filtering for instances where this ratio is less 
than or equal to 0.85, indicating a reduction in actual sales by 15% or more.
The result is sorted by date_analyzed.
*/
select 
    hol.holiday_name,
    hol.date_analyzed,
    hol.day_of_week,
    act.actual_sales_dollars,
    exp.expected_sales_dollars,
    round(act.actual_sales_dollars::numeric / exp.expected_sales_dollars, 2) 
        as ratio_actual_expected
from 
    holidays_tmp hol
join
    actual_sales_tmp act on hol.date_analyzed = act.sale_date
join
    expected_sales_tmp exp ON hol.dow = exp.dow
where 
    act.actual_sales_dollars / exp.expected_sales_dollars <= 0.85
order by 
    hol.date_analyzed;

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,holiday_name,date_analyzed,day_of_week,actual_sales_dollars,expected_sales_dollars,ratio_actual_expected
0,New Year's Day,2020-01-01,Wednesday,133776,263256,0.51
1,MLK Day,2020-01-17,Friday,127092,252522,0.50
2,MLK Day,2020-01-18,Saturday,135204,373490,0.36
3,MLK Day,2020-01-19,Sunday,130368,357482,0.36
4,MLK Day,2020-01-20,Monday,130740,253225,0.52
5,President's Day,2020-02-14,Friday,133452,252522,0.53
6,President's Day,2020-02-15,Saturday,132096,373490,0.35
7,President's Day,2020-02-16,Sunday,132180,357482,0.37
8,President's Day,2020-02-17,Monday,135228,253225,0.53
9,Easter,2020-04-12,Sunday,136164,357482,0.38
